<a href="https://colab.research.google.com/github/present42/PyTorchPractice/blob/main/Fluent_Python_ch8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

11:34 - 12:09

In [ ]:
%load_ext autoreload
%autoreload 2

## About Gradaul Typing

Properties:
 1. Optional
 2. Does not catch type erros at runtime
 3. Does not enhance performance

In [ ]:
def show_count(count, word):
  if count == 1:
    return f'1 {word}'
  count_str = str(count) if count else 'no'
  return f'{count_str} {word}s'

In [ ]:
show_count(99, 'bird')

'99 birds'

In [ ]:
show_count(1, 'bird')

'1 bird'

In [ ]:
show_count(0, 'bird')

'no birds'

In [ ]:
!pip install mypy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 82.4 MB/s eta 0:00:00


In [ ]:
!mypy messages.py

Success: no issues found in 1 source file


In [ ]:
from pytest import mark

from messages import show_count

@mark.parametrize('qty, expected', [
    (1, '1 part'),
    (2, '2 parts'),
])
def test_show_count(qty, expected):
  got = show_count(qty, 'part')
  assert got == expected

def test_show_count_zero():
  got = show_count(0, 'part')
  assert got == 'no parts'

In [ ]:
!mypy --disallow-untyped-defs messages_test.py

messages.py:1: error: Function is missing a type annotation  [no-untyped-def]
messages_test.py:9: error: Function is missing a type annotation  [no-untyped-def]
messages_test.py:13: error: Function is missing a return type annotation  [no-untyped-def]
messages_test.py:13: note: Use "-> None" if function does not return a value
Found 3 errors in 2 files (checked 1 source file)


In [ ]:
!mypy messages_test.py

Success: no issues found in 1 source file


In [ ]:
from typing import Optional

def show_count(count: int, singular: str, plural: Optional[str] = None)

## Types Are Defined by Supported Operations

In [ ]:
from collections import abc
def double(x: abc.Sequence):
  return x * 2

A type checker will reject that code (`double(x)` wher `x` has type `str`, `tuple`, ..., `array`) because the type checker only cares about what is explicitly declared, and `abc.Sequence` has no `__mul__`.

### Duck Typing
 - The view adopted by SmallTalk as well as Python, Javascript, and Ruby. In practice, it doesn't matter what the declared type of the object is, only what operations it actually supports. If I can invoke `birdie.quack()`, then the `birdie` is a duck in this context. By definition, duck typing is only enforced at runtime.

### Nominal Typing
 - Adopted by C++, Java and C#, supported by annotated Python. Objects and vars have types but objects only exist at runtime and the type checker only cares about the source code where variables are annotated with type hints.

In [ ]:
!mypy birds.py

birds.py:15: error: "Bird" has no attribute "quack"  [attr-defined]
Found 1 error in 1 file (checked 1 source file)


In [ ]:
# daffy.py
from birds import *

daffy = Duck()
alert(daffy)
alert_duck(daffy)
alert_bird(daffy)

In [ ]:
!mypy daffy.py

birds.py:15: error: "Bird" has no attribute "quack"  [attr-defined]
Found 1 error in 1 file (checked 1 source file)


In [ ]:
!python daffy.py

Quack!
Quack!
Quack!


In [ ]:
# woody.py

from birds import *

woody = Bird()
alert(woody)
alert_duck(woody)
alert_bird(woody)

In [ ]:
!mypy woody.py

birds.py:15: error: "Bird" has no attribute "quack"  [attr-defined]
woody.py:7: error: Argument 1 to "alert_duck" has incompatible type "Bird"; expected "Duck"  [arg-type]
Found 2 errors in 2 files (checked 1 source file)


In [ ]:
!python woody.py

Traceback (most recent call last):
  File "/content/woody.py", line 6, in <module>
    alert(woody)
  File "/content/birds.py", line 9, in alert
    birdie.quack()
AttributeError: 'Bird' object has no attribute 'quack'


9:48 -

In [ ]:
!pip install mypy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 72.0 MB/s eta 0:00:00


In [ ]:
!mypy double_object.py

Success: no issues found in 1 source file


* Traditional OOP
  - Liskov Property: If an object of type `T2` substitutes an object of type `T1` and the program still behaves correctly, then `T2` is subtype of `T1`.
* Gradual Type System
  - consistent-with relationship
  - which applies wherever subtype-of applies, with special provision for type `Any`

  - Rules
    1. Given `T1` and a subtype `T2`, then `T2` is consistent with `T1` (Liskov substitution)
    2. Every type is consistent with `Any`: you can pass objects of every type to an argument declared of type `Any`.
    3. `Any` is consistent-with every type: you can always pass an object of type `Any` where an argument of another type is expected.

## Optional and Union Types

The construct `Optional[str]` is actually a shortcut for `Union[str, None]`

In [ ]:
from typing import Optional

def show_count(count: int, signular: str, plural: Optional[str] = None) -> str:


## Generic Collection

In [ ]:
def tokenize(text: str) -> list[str]:
  return text.upper().split()

## Tuple Types

Three ways to annotate tuple types:
 1. Tuples as records
 2. Tuples as records with named fields
 3. Tuples as immutable sequences

1. Tuples as records
: use `tuple` built-in and declare the types of the fields within `[]`

In [ ]:
!pip install geolib

In [ ]:
from geolib import geohash as gh # type: ignore
PRECISION = 9

def geohash(lat_lon: tuple[float, float]) -> str:
  return gh.encode(*lat_lon, PRECISION)

In [ ]:
shanghai = 31.2304, 121.4737
geohash(shanghai)

'wtw3sjq6q'

2. Tuples as records with named fields

In [ ]:
from typing import NamedTuple

from geolib import geohash as gh # type: ignore

PRECISION = 9

# NamedTuple is a factory for `tuple` subclasses
# so `Coordinate` is consistent-with tuple[float, float]
class Coordinate(NamedTuple):
  lat: float
  lon: float

def geohash(lat_lon: Coordinate) -> str:
  return gh.encode(*lat_lon, PRECISION)

geohash(shanghai)

In [ ]:
!mypy coordinates_named.py

coordinates_named.py:15: error: Argument 1 to "geohash" has incompatible type "tuple[float, float]"; expected "Coordinate"  [arg-type]
Found 1 error in 1 file (checked 1 source file)


In [ ]:
geohash(shanghai)

'wtw3sjq6q'

3. Tuples as immutable Sequences

To annotate tuples of unspecified length that are used as immutable lists, you must specify a single type, followed by a comma and ...

In [ ]:
# ex:
tuple[int, ...] # is a tuple with int items

In [ ]:
from collections.abc import Sequence

def columnize(
    sequence: Sequence[str], num_columns: int = 0
) -> list[tuple[str, ...]]:
  if num_columns == 0:
    num_columns = round(len(sequence) ** 0.5)
  num_rows, remainder = divmod(len(sequence), num_columns)
  num_rows += bool(remainder)
  return [tuple(sequence[i::num_rows]) for i in range(num_rows)]

animals = 'drake fawn heron ibex koala lynx tahr xerus yak zapus'.split()
table = columnize(animals)
table

[('drake', 'koala', 'yak'),
 ('fawn', 'lynx', 'zapus'),
 ('heron', 'tahr'),
 ('ibex', 'xerus')]